In [45]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import ReadVitalData
import ReadDrugsData
import pandas as pd
from functools import reduce

In [6]:
vital = ReadVitalData.vital
mydi = ReadDrugsData.di
klients = ReadVitalData.klients


In [70]:
myklient = klients[133]
df = vital[vital.klientid2 == myklient]

#Blutzucker
bz = df[df.bezeichnung.isin(["Blutzucker", "Blutzucker mmol/l"])]
num = pd.to_numeric(bz.wert,errors='coerce')
bz["wert"] = num
bz.sort_values(by='datum',inplace=True)
bz = bz[["datum","wert"]]
bz.columns = ["datum", "blutzucker"]
bz = bz.set_index("datum")
bz = bz.resample('1d').mean()

#Gewicht

gw = df[df.bezeichnung == "Gewicht"]
num = [float(str(x).replace(",",".")) for x in gw.wert]
gw["wert"] = num
gw.sort_values(by='datum',inplace=True)
gw = gw[["datum","wert"]]
gw.columns = ["datum", "gewicht"]
gw = gw.set_index("datum")
gw.index = [d.date() for d in gw.index]

#bmi
bmi = df[df.bezeichnung == "BMI"]
num = [float(str(x).replace(",",".")) for x in bmi.wert]
bmi["wert"] = num
bmi.sort_values(by='datum',inplace=True)
bmi = bmi[["datum","wert"]]
bmi.columns = ["datum", "bmi"]
bmi = bmi.set_index("datum")
bmi.index = [d.date() for d in bmi.index]

#Blutdruck

bd = df[df.bezeichnung == "Blutdruck"]
werte = list(bd.wert.apply(str))
sys = [float(x.split("/")[0]) if len(x)>1 else np.NaN for x in werte]
dia = [float(x.split("/")[1]) if len(x)>1 else np.NaN for x in werte]

bd["sys"] = sys
bd["dia"] = dia
bd.sort_values(by='datum',inplace=True)
syst = bd[["datum","sys"]]
syst = syst.set_index("datum")
syst = syst.resample('1d').mean()

dia = bd[["datum","dia"]]
dia = dia.set_index("datum")
dia = dia.resample('1d').mean()

#Medikamente
mdf = mydi[mydi.klientid2 == myklient]
medi = list(mdf.inhaltsstoff.unique())

a = medi[0]
adf = mdf[mdf.inhaltsstoff == a]
adf = adf[["datum","eingenommene_menge"]]
adf.columns = ["datum",a]
adf.sort_values(by='datum',inplace=True)
adf.set_index("datum",inplace=True)
adf = adf.resample('1d').sum()

for b in medi[1::]:

    bdf = mdf[mdf.inhaltsstoff == b]
    bdf = bdf[["datum","eingenommene_menge"]]
    bdf.columns = ["datum",b]
    bdf.sort_values(by='datum',inplace=True)
    bdf.set_index("datum",inplace=True)
    bdf = bdf.resample('1d').sum()

    adf = adf.join(bdf, lsuffix='_caller', rsuffix='_other',how='outer').fillna(0)

adf = adf.loc[:, (adf != 0).any(axis=0)]

dfs = [adf, bz, syst, dia, gw, bmi]
final_df = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), dfs)
final_df.resample('1w').mean()

<ipython-input-70-bafa7c265aef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bz["wert"] = num
<ipython-input-70-bafa7c265aef>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bz.sort_values(by='datum',inplace=True)
<ipython-input-70-bafa7c265aef>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gw["wert"] = num
<ipytho

,Ramipril,"Pantoprazol natrium-1,5-Wasser",Insulin glargin,Simvastatin,Colecalciferol,Torasemid,Natriumchlorid,Natriumhydrogencarbonat,Macrogol 3350,Kaliumchlorid,...,Apixaban,Allopurinol,Melperon hydrochlorid,Insulin glulisin,Rivaroxaban,blutzucker,sys,dia,gewicht,bmi
2017-02-19,11.250000,101.497500,16200.000000,0.0,0.056250,11.250000,0.0,0.0,0.0,0.0,...,0.000000,225.000000,1125.000000,14850.000000,22.500000,203.833333,125.000000,65.000000,61.3,27.24
2017-02-26,15.000000,135.330000,18514.285714,0.0,0.075000,15.000000,0.0,0.0,0.0,0.0,...,0.000000,300.000000,1125.000000,18128.571429,30.000000,208.619048,121.428571,58.571429,NaN,NaN
2017-03-05,15.000000,135.330000,16200.000000,0.0,0.075000,15.000000,0.0,0.0,0.0,0.0,...,0.000000,300.000000,1125.000000,27900.000000,30.000000,228.833333,120.000000,60.000000,65.7,29.20
2017-03-12,15.000000,135.330000,13885.714286,0.0,0.075000,15.000000,0.0,0.0,0.0,0.0,...,0.000000,300.000000,964.285714,18128.571429,30.000000,220.761905,120.000000,66.666667,NaN,NaN
2017-03-19,12.857143,115.997143,16200.000000,0.0,0.064286,12.857143,0.0,0.0,0.0,0.0,...,0.000000,257.142857,1125.000000,17100.000000,25.714286,214.809524,122.000000,65.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-28,0.000000,115.997143,0.000000,0.0,0.064286,0.000000,0.0,0.0,0.0,0.0,...,13.928571,0.000000,1017.857143,15171.428571,0.000000,170.476190,NaN,NaN,65.5,29.11
2021-12-05,0.000000,135.330000,0.000000,0.0,0.075000,0.000000,0.0,0.0,0.0,0.0,...,15.000000,0.000000,1125.000000,13500.000000,0.000000,164.630952,NaN,NaN,65.7,29.20
2021-12-12,0.000000,96.664286,0.000000,0.0,0.053571,0.000000,0.0,0.0,0.0,0.0,...,12.857143,0.000000,910.714286,12342.857143,0.000000,172.940476,NaN,NaN,NaN,NaN
2021-12-19,0.000000,115.997143,0.000000,0.0,0.064286,0.000000,0.0,0.0,0.0,0.0,...,13.928571,0.000000,1017.857143,11700.000000,0.000000,178.380952,NaN,NaN,NaN,NaN


In [ ]:

#interpolate(method = 'linear')

